In [85]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime as dt
from datetime import timedelta
from darksky import forecast
key = "e202a3b89ed3182ce3d86552661df096" #darksky key
from sklearn.linear_model import LogisticRegression

In [18]:
# US fires data - has the cause of fire
# https://www.fs.usda.gov/rds/archive/catalog/RDS-2013-0009.4
# Metadata - https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.4/_metadata_RDS-2013-0009.4.html
us_fires1 = pd.read_csv("us_fires_1.csv")
us_fires2 = pd.read_csv("us_fires_2.csv")
us_fires3 = pd.read_csv("us_fires_3.csv")
us_fires4 = pd.read_csv("us_fires_4.csv")
us_fires5 = pd.read_csv("us_fires_5.csv")
us_fires6 = pd.read_csv("us_fires_6.csv")
us_fires7 = pd.read_csv("us_fires_7.csv")

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (11,12,16,17,18,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (10,11,12,13,14,15,18,25,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (11,12,14,18,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interact

In [22]:
# Concatenate all us_fires data
pd.set_option('display.max_columns', None)
us_fires_all = pd.concat([us_fires1, us_fires2], ignore_index=True)
us_fires_all = pd.concat([us_fires_all, us_fires3], ignore_index=True)
us_fires_all = pd.concat([us_fires_all, us_fires4], ignore_index=True)
us_fires_all = pd.concat([us_fires_all, us_fires5], ignore_index=True)
us_fires_all = pd.concat([us_fires_all, us_fires6], ignore_index=True)
us_fires_all = pd.concat([us_fires_all, us_fires7], ignore_index=True)

,objectid,fod_id,fpa_id,source_system_type,source_system,nwcg_reporting_agency,nwcg_reporting_unit_id,nwcg_reporting_unit_name,source_reporting_unit,source_reporting_unit_name,local_fire_report_id,local_incident_id,fire_code,fire_name,ics_209_incident_number,ics_209_name,mtbs_id,mtbs_fire_name,complex_name,fire_year,discovery_date,discovery_doy,discovery_time,stat_cause_code,stat_cause_descr,cont_date,cont_doy,cont_time,fire_size,fire_size_class,latitude,longitude,owner_code,owner_descr,state,county,fips_code,fips_name
0,1,1.0,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,Plumas National Forest,1,PNF-47,BJ8K,FOUNTAIN,NaN,NaN,NaN,NaN,NaN,2005,2005-02-02T00:00:00,33,1300.0,9,Miscellaneous,2005-02-02T00:00:00,33.0,1730.0,0.10,A,40.036944,-121.005833,5,USFS,CA,63,63.0,Plumas
1,2,2.0,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,13,13,AAC0,PIGEON,NaN,NaN,NaN,NaN,NaN,2004,2004-05-12T00:00:00,133,845.0,1,Lightning,2004-05-12T00:00:00,133.0,1530.0,0.25,A,38.933056,-120.404444,5,USFS,CA,61,61.0,Placer
2,3,3.0,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,27,021,A32W,SLACK,NaN,NaN,NaN,NaN,NaN,2004,2004-05-31T00:00:00,152,1921.0,5,Debris Burning,2004-05-31T00:00:00,152.0,2024.0,0.10,A,38.984167,-120.735556,13,STATE OR PRIVATE,CA,17,17.0,El Dorado
3,4,4.0,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,43,6,NaN,DEER,NaN,NaN,NaN,NaN,NaN,2004,2004-06-28T00:00:00,180,1600.0,1,Lightning,2004-07-03T00:00:00,185.0,1400.0,0.10,A,38.559167,-119.913333,5,USFS,CA,3,3.0,Alpine
4,5,5.0,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,44,7,NaN,STEVENOT,NaN,NaN,NaN,NaN,NaN,2004,2004-06-28T00:00:00,180,1600.0,1,Lightning,2004-07-03T00:00:00,185.0,1200.0,0.10,A,38.559167,-119.933056,5,USFS,CA,3,3.0,Alpine


In [28]:
us_fires_all.head()
us_fires_all.tail()

,objectid,fod_id,fpa_id,source_system_type,source_system,nwcg_reporting_agency,nwcg_reporting_unit_id,nwcg_reporting_unit_name,source_reporting_unit,source_reporting_unit_name,local_fire_report_id,local_incident_id,fire_code,fire_name,ics_209_incident_number,ics_209_name,mtbs_id,mtbs_fire_name,complex_name,fire_year,discovery_date,discovery_doy,discovery_time,stat_cause_code,stat_cause_descr,cont_date,cont_doy,cont_time,fire_size,fire_size_class,latitude,longitude,owner_code,owner_descr,state,county,fips_code,fips_name
1880460,1880461,300348000.0,2015CAIRS29019636,NONFED,ST-CACDF,ST/C&L,USCASHU,Shasta-Trinity Unit,CASHU,Shasta-Trinity Unit,591814,009371,NaN,ODESSA 2,NaN,NaN,NaN,NaN,NaN,2015,2015-09-26T00:00:00,269,1726.0,13,Missing/Undefined,2015-09-26T00:00:00,269.0,1843.0,0.01,A,40.481637,-122.389375,13,STATE OR PRIVATE,CA,NaN,NaN,NaN
1880461,1880462,300348000.0,2015CAIRS29217935,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,569419,000366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015-10-05T00:00:00,278,126.0,9,Miscellaneous,NaN,NaN,NaN,0.20,A,37.617619,-120.938570,12,MUNICIPAL/LOCAL,CA,NaN,NaN,NaN
1880462,1880463,300348000.0,2015CAIRS28364460,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,574245,000158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015-05-02T00:00:00,122,2052.0,13,Missing/Undefined,NaN,NaN,NaN,0.10,A,37.617619,-120.938570,12,MUNICIPAL/LOCAL,CA,NaN,NaN,NaN
1880463,1880464,300348000.0,2015CAIRS29218079,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,570462,000380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015-10-14T00:00:00,287,2309.0,13,Missing/Undefined,NaN,NaN,NaN,2.00,B,37.672235,-120.898356,12,MUNICIPAL/LOCAL,CA,NaN,NaN,NaN
1880464,1880465,300348000.0,2015CAIRS26733926,NONFED,ST-CACDF,ST/C&L,USCABDU,San Bernardino Unit,CABDU,CDF - San Bernardino Unit,535436,003225,NaN,BARKER BL BIG_BEAR_LAKE_,NaN,NaN,NaN,NaN,NaN,2015,2015-03-14T00:00:00,73,2128.0,9,Miscellaneous,NaN,NaN,NaN,0.10,A,34.263217,-116.830950,13,STATE OR PRIVATE,CA,NaN,NaN,NaN


In [27]:
len(us_fires_all)

1880465

In [115]:
train = us_fires_all[['objectid',
                      'fod_id',
                      'fpa_id',
                      'fire_year',
                      'discovery_date',
                      'discovery_doy',
                      'stat_cause_code',
                      'stat_cause_descr',
                      'fire_size',
                      'fire_size_class',
                      'latitude',
                      'longitude',
                      'state']]
train.head()

,objectid,fod_id,fpa_id,fire_year,discovery_date,discovery_doy,stat_cause_code,stat_cause_descr,fire_size,fire_size_class,latitude,longitude,state
0,1,1.0,FS-1418826,2005,2005-02-02T00:00:00,33,9,Miscellaneous,0.10,A,40.036944,-121.005833,CA
1,2,2.0,FS-1418827,2004,2004-05-12T00:00:00,133,1,Lightning,0.25,A,38.933056,-120.404444,CA
2,3,3.0,FS-1418835,2004,2004-05-31T00:00:00,152,5,Debris Burning,0.10,A,38.984167,-120.735556,CA
3,4,4.0,FS-1418845,2004,2004-06-28T00:00:00,180,1,Lightning,0.10,A,38.559167,-119.913333,CA
4,5,5.0,FS-1418847,2004,2004-06-28T00:00:00,180,1,Lightning,0.10,A,38.559167,-119.933056,CA


In [116]:
train.isnull().sum()

objectid            0
fod_id              0
fpa_id              0
fire_year           0
discovery_date      0
discovery_doy       0
stat_cause_code     0
stat_cause_descr    0
fire_size           0
fire_size_class     0
latitude            0
longitude           0
state               0
dtype: int64

In [117]:
# Regex extract date
train['discovery_date'] = train['discovery_date'].str.extract(r'(\d{4}-\d{2}-\d{2})')
train.head()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,objectid,fod_id,fpa_id,fire_year,discovery_date,discovery_doy,stat_cause_code,stat_cause_descr,fire_size,fire_size_class,latitude,longitude,state
0,1,1.0,FS-1418826,2005,2005-02-02,33,9,Miscellaneous,0.10,A,40.036944,-121.005833,CA
1,2,2.0,FS-1418827,2004,2004-05-12,133,1,Lightning,0.25,A,38.933056,-120.404444,CA
2,3,3.0,FS-1418835,2004,2004-05-31,152,5,Debris Burning,0.10,A,38.984167,-120.735556,CA
3,4,4.0,FS-1418845,2004,2004-06-28,180,1,Lightning,0.10,A,38.559167,-119.913333,CA
4,5,5.0,FS-1418847,2004,2004-06-28,180,1,Lightning,0.10,A,38.559167,-119.933056,CA


In [118]:
# Convert to timestamp
train['discovery_date'] = train['discovery_date'].apply(lambda x: dt.strptime(x, '%Y-%m-%d'))
type(train['discovery_date'][1])
# Convert back to string
# train['discovery_date'] = train['discovery_date'].astype(str)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


pandas._libs.tslibs.timestamps.Timestamp

In [119]:
# Consolidate years - overall dataset is 1992 - 2015
len(train)
train.loc[train['fire_year'].idxmax()]

objectid                        1789012
fod_id                        3.002e+08
fpa_id                       FS-6341424
fire_year                          2015
discovery_date      2015-08-14 00:00:00
discovery_doy                       226
stat_cause_code                       1
stat_cause_descr              Lightning
fire_size                           0.1
fire_size_class                       A
latitude                        45.9342
longitude                      -113.021
state                                MT
Name: 1789011, dtype: object

In [121]:
test_recent = train[(train['discovery_date'] > '2015-01-01') & (train['discovery_date'] < '2015-08-14')]
test = test_recent.sample(5000)

train_recent = train[(train['discovery_date'] > '2014-01-01') & (train['discovery_date'] < '2014-12-31')]
train = train_recent.sample(10000)

10000

In [ ]:
# Populating with Dark Sky API 
# train['Lat'] = np.zeros(len(train))
# train['Long'] = np.zeros(len(train))
train['Month'] = np.zeros(len(train))
train['Temp'] = np.zeros(len(train))
train['WindSpd'] = np.zeros(len(train))
train['WindGust'] = np.zeros(len(train))
train['Humidity'] = np.zeros(len(train))
train['Precipitation'] = np.zeros(len(train))
for i in np.arange(0,len(train)):
#     #Convert Lat/Long to 2 separate variables
#     latlong = (((train['Lat/Long'].iloc[i]).strip()).strip('][')).split(",")
#     train['Lat'].iloc[i] = float(latlong[0])
#     train['Long'].iloc[i] = float(latlong[1])
    #Add "Month" variable
#     timestamp = train['discovery_date'].iloc[i].split("/")
#     train['Month'].iloc[i] = float(timestamp[0])+(int(timestamp[1])/31)
    #Add weather variables
#     t = dt(int(timestamp[2]),int(timestamp[0]),int(timestamp[1]),12).isoformat()
    try:
        weather = forecast(key,train['latitude'].iloc[i],train['longitude'].iloc[i],time=t)
        train['Temp'].iloc[i] = weather.temperature
        train['WindSpd'].iloc[i] = weather.windSpeed
        train['WindGust'].iloc[i] = weather.windGust
        train['Humidity'].iloc[i] = weather.humidity
        train['Precipitation'].iloc[i] = weather.precipIntensity
    except:
        #weather = forecast(key,round(train['Lat'].iloc[i])+1,round(train['Long'].iloc[i])+1,time=t)
        train['Temp'].iloc[i] = "BAD RESPONSE"
        
train.to_csv('training_data.csv')

In [127]:
# Populating with Dark Sky API 
# train['Lat'] = np.zeros(len(train))
# train['Long'] = np.zeros(len(train))
test['Month'] = np.zeros(len(test))
test['Temp'] = np.zeros(len(test))
test['WindSpd'] = np.zeros(len(test))
test['WindGust'] = np.zeros(len(test))
test['Humidity'] = np.zeros(len(test))
test['Precipitation'] = np.zeros(len(test))
for i in np.arange(0,len(test)):
#     #Convert Lat/Long to 2 separate variables
#     latlong = (((train['Lat/Long'].iloc[i]).strip()).strip('][')).split(",")
#     train['Lat'].iloc[i] = float(latlong[0])
#     train['Long'].iloc[i] = float(latlong[1])
    #Add "Month" variable
#     timestamp = train['discovery_date'].iloc[i].split("/")
#     train['Month'].iloc[i] = float(timestamp[0])+(int(timestamp[1])/31)
    #Add weather variables
#     t = dt(int(timestamp[2]),int(timestamp[0]),int(timestamp[1]),12).isoformat()
    try:
        weather = forecast(key,test['latitude'].iloc[i],test['longitude'].iloc[i],time=t)
        test['Temp'].iloc[i] = weather.temperature
        test['WindSpd'].iloc[i] = weather.windSpeed
        test['WindGust'].iloc[i] = weather.windGust
        test['Humidity'].iloc[i] = weather.humidity
        test['Precipitation'].iloc[i] = weather.precipIntensity
    except:
        #weather = forecast(key,round(train['Lat'].iloc[i])+1,round(train['Long'].iloc[i])+1,time=t)
        test['Temp'].iloc[i] = "BAD RESPONSE"
        
test.to_csv('testing_data.csv')

In [128]:
# Read in training and testing
training_data = pd.read_csv("training_data.csv")
testing_data = pd.read_csv("testing_data.csv")

In [132]:
training_data

,Unnamed: 0,objectid,fod_id,fpa_id,fire_year,discovery_date,discovery_doy,stat_cause_code,stat_cause_descr,fire_size,fire_size_class,latitude,longitude,state,Month,Temp,WindSpd,WindGust,Humidity,Precipitation
0,1763044,1763045,300106000.0,SFO-2014TXLFDX538725,2014,2014-07-02,183,5,Debris Burning,1.00,B,29.482184,-95.288704,TX,0.0,BAD RESPONSE,0.0,0.0,0.0,0.0
1,1756167,1756168,300098000.0,SFO-2014NYNYSNYS-2014-0040,2014,2014-04-18,108,5,Debris Burning,1.50,B,42.261600,-77.963800,NY,0.0,BAD RESPONSE,0.0,0.0,0.0,0.0
2,1780419,1780420,300135000.0,SFO-2014MSMFC35055,2014,2014-08-26,238,9,Miscellaneous,7.00,B,30.732630,-88.545920,MS,0.0,BAD RESPONSE,0.0,0.0,0.0,0.0
3,1744172,1744173,300084000.0,SFO-2014TXLFDX531058,2014,2014-06-20,171,2,Equipment Use,20.00,C,28.335813,-98.022079,TX,0.0,BAD RESPONSE,0.0,0.0,0.0,0.0
4,1754149,1754150,300096000.0,SFO-2014NCNCSNCST-034-20140043,2014,2014-03-27,86,5,Debris Burning,2.00,B,36.201915,-80.194809,NC,0.0,BAD RESPONSE,0.0,0.0,0.0,0.0
5,1739188,1739189,300077000.0,SFO-2014NY4040NY4040-2014-36,2014,2014-05-12,132,1,Lightning,0.10,A,42.040200,-75.515800,NY,0.0,BAD RESPONSE,0.0,0.0,0.0,0.0
6,1759011,1759012,300102000.0,SFO-2014SCSCS15FF0084,2014,2014-08-27,239,5,Debris Burning,5.50,B,34.033886,-80.220520,SC,0.0,BAD RESPONSE,0.0,0.0,0.0,0.0
7,1746858,1746859,300087000.0,SFO-2014MSMFC34672,2014,2014-05-06,126,4,Campfire,5.00,B,31.098751,-88.536911,MS,0.0,BAD RESPONSE,0.0,0.0,0.0,0.0
8,1754304,1754305,300096000.0,SFO-2014MSMFC10581,2014,2014-03-31,90,5,Debris Burning,1.00,B,32.779379,-88.571980,MS,0.0,BAD RESPONSE,0.0,0.0,0.0,0.0
9,1735462,1735463,300026000.0,W-670215,2014,2014-07-25,206,9,Miscellaneous,0.10,A,43.292800,-100.613300,SD,0.0,BAD RESPONSE,0.0,0.0,0.0,0.0


In [ ]:
x_train = train[!((train['discovery_date'] > '2012-01-01') & (train['discovery_date'] < '2015-08-14'))]
x_test = train[(train['discovery_date'] > '2012-01-01') & (train['discovery_date'] < '2015-08-14')]

In [ ]:
y_train = x_train['fire_size_class']
y_test = x_test['fire_size_class']

In [11]:
#model
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: '[37.7393, -121.85663]'

In [12]:
#predict
predictions = logisticRegr.predict(x_test)

NameError: name 'x_test' is not defined

In [ ]:
# Use score method to get accuracy of model
score = logisticRegr.score(x_test, y_test)
print(score)